# RNN에서 Seq2Seq로, 그리고 어텐션 메커니즘 + 임베이딩

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dot, Activation, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

cal_len = 15
# 문자와 인덱스 매핑
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
num_classes = len(chars)

char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for idx, char in enumerate(chars)}
# 데이터 생성 함수
def generate_data(num_samples):
    X = []
    y = []
    for _ in range(num_samples):
        sequence = np.random.choice(list(chars), cal_len)
        X.append([char_to_index[char] for char in sequence]) 
        y.append([char_to_index[char] for char in sequence[::-1]])
    return np.array(X), np.array(y)

# 학습 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)

In [6]:
# 인코더 정의
encoder_inputs = Input(shape=(cal_len, 1))
# 임베딩 추가
encoder_embedding = Embedding(input_dim=num_classes, output_dim=64, input_length=cal_len)(encoder_inputs)
encoder = LSTM(128, return_sequences=True,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# 디코더 정의
decoder_inputs = Input(shape=(cal_len, 1))
# 임베딩 추가
decoder_embedding = Embedding(input_dim=num_classes, output_dim=64, input_length=cal_len)(decoder_inputs)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# 어텐션
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)
context = Dot(axes=[2, 1])([attention, encoder_outputs])

# 디코더 출려과 컨텍스트 결합
decoder_combined_context = Concatenate(axis=-1)([decoder_outputs, context])

# 출력 레이어
decoder_dense = Dense(52, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

# 모데정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:

# 입력 데이터와 타겟 데이터를 동일하게 맞춤
decoder_input_data = np.zeros_like(X)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X, decoder_input_data], y, epochs=50, validation_split=0.2, callbacks=[early_stopping])

def predict_reverse(input_sequence):
    input_sequence = np.array([char_to_index[char] for char in input_sequence]).reshape((1, cal_len, 1))    
    decoder_input = np.zeros((1, cal_len, 1))
    predicted_sequence = model.predict ([input_sequence, decoder_input])
    predicted_indices = np.argmax(predicted_sequence, axis=-1). reshape((cal_len,))
    return "".join([index_to_char[idx] for idx in predicted_indices])
    # return np.argmax(predicted_sequence, axis=-1). reshape((10,))


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.0387 - loss: 3.8063 - val_accuracy: 0.1409 - val_loss: 2.8689
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1929 - loss: 2.6396 - val_accuracy: 0.3265 - val_loss: 2.1754
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.3480 - loss: 2.1275 - val_accuracy: 0.4213 - val_loss: 1.8770
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.4691 - loss: 1.8036 - val_accuracy: 0.5415 - val_loss: 1.6220
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.5618 - loss: 1.5672 - val_accuracy: 0.6031 - val_loss: 1.4370
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6317 - loss: 1.3820 - val_accuracy: 0.6947 - val_loss: 1.2643
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.6866 - loss: 1.2431 - val_accuracy: 0.7157 - val_loss: 1.1614
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.7237 - loss: 1.1469 - val_accu

In [8]:
#검증용 문자열 리스트 (5글자짜리 30개)
test_strings = ["".join(np.random.choice(list(chars), cal_len)) for _ in range(30)]
# 실제 거꾸로 된 문자열 리스트
expected_outputs = [s[::-1] for s in test_strings]

correct_predictions = 0
total_predictions = len(test_strings)

for i, test_string in enumerate(test_strings):
    predicted_output = predict_reverse(test_string)
    is_correct = predicted_output == expected_outputs[i]
    
    if is_correct:
        correct_predictions += 1
        
    print(f"입력: {test_string}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {expected_outputs[i]}")
    print(f"정확 여부: {'맞음' if is_correct else '틀림'}\n")

accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
입력: RYblxIucAvmbJyO
예측된 출력: OyJbmvAcuIxlbYR
실제 출력: OyJbmvAcuIxlbYR
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: eEyxTvejxyikkNa
예측된 출력: aNkkiyxjevTxyEe
실제 출력: aNkkiyxjevTxyEe
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: JIqaZfyeZOXCJfT
예측된 출력: TfJCXOZeyfZaqIJ
실제 출력: TfJCXOZeyfZaqIJ
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
입력: uMtFEVCYMifTKXY
예측된 출력: YXKTfiMYCVEFtMu
실제 출력: YXKTfiMYCVEFtMu
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: PZgsdWBgfHYfvCU
예측된 출력: UCvfYHfgBWdsgZP
실제 출력: UCvfYHfgBWdsgZP
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: dRDsoycCUkHfPpU
예측된 출력: UpPfHkUCcyosDRd
실제 출력: UpPfHkUCcyosDRd
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
입력: IETEHBFEhqFVoFR
예측된 출력: RFoVFqhEFBHETEI
실제 출력: RFoVFqhEFBHETEI
정확 여부: 맞음

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: AbJavjAICsDKdPL
예측된 출력: LPdKDsCIAjvaJaA
실제 출력: LPdKDsCIAjvaJbA
정확 여부: 틀림

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
입력: jBwMdZmCUkcUvqM
예측된 출력: MqvUckUCmZdM